In [1]:
##Model Training

Model training

In [5]:
import pandas as pd
df = pd.read_csv('gemstone.csv')

In [3]:
#df = df.drop(labels='id',axis=1)

In [6]:
## independent and dependent features
x = df.drop(labels='id',axis=1)
y = df[['price']]

In [7]:
# define which columns should be ordinal encoded
categorical_cols = x.select_dtypes(include='object').columns
numerical_cols = y.select_dtypes(exclude='object').columns

In [8]:
# define the custom ranking for each ordinal variable
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
claritY_categories = ['Il','SI2','SI1','VS2','VS1','VVS2', 'VVS1','IF']

In [ ]:
from sklearn.preprocessing import 